## This notebook will be used for the Coursera Capstone project

In [2]:
import pandas as pd
import numpy as np

In [3]:
import requests
from bs4 import BeautifulSoup

Parsing the html file source from the wikipedia  to scrap the table

In [4]:
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.content, 'lxml')

rows = soup.find('table').find_all('tr')
cells = []
for row in rows:
    cells.append(row.find_all(['th','td']))

In [5]:
cells_str = []
for row in cells:
    r = []
    for cell in row:
        cell_string = cell.string.replace('\n','').replace(' / ',', ')
        if cell_string=='' or cell_string=='Not assigned':
            r.append(None)
        else:
            r.append(cell_string)
    cells_str.append(r)

Converting scraped data into Pandas dataframe, cleaning the data

In [6]:
df = pd.DataFrame(cells_str)
df.columns = df.loc[0,:]
df.drop([0],inplace = True)
df.dropna(subset=['Borough'], inplace = True)

Checking for missing values or duplicated values in the postal code column

In [7]:
df.describe()

,Postal code,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M4W,North York,Downsview
freq,1,24,4


In [8]:
df.isna().sum()

0
Postal code     0
Borough         0
Neighborhood    0
dtype: int64

In [12]:
df.to_csv('neighborhoods.csv')

Importing coordinates data and merging with the created datafame

In [ ]:
latlng_df = pd.read_csv('https://cocl.us/Geospatial_data')
latlng_df.head()

In [ ]:
df = df.merge(latlng_df, how='inner',left_on = ['Postal code'], right_on = ['Postal Code']).drop(['Postal Code'], axis = 1)

In [ ]:
df.describe(include = 'all')

Import surrounding venues for each neighborhood into a new dataframe

In [ ]:
CLIENT_ID = 'BWPEENWH3CWY3SKJT24EFITHI3MKSF1NAFC1IVBSC5DDYKEH' # your Foursquare ID
CLIENT_SECRET = 'EIQE2K2CDDBQSCAZ5KOLN2PHONXB31RJLBLGEPKS0VJ20X25' # your Foursquare Secret
VERSION = '20200420'
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
     # query close venues to the Foursquare API for each neighborhood, return a new dataframe with all the returned venues,
     # their coordinates and main category   
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_neighborhoods = df[df['Borough'].str.contains('Toronto')]
toronto_neighborhoods.head() #filtered dataframe with only Toronto neighborhoods

In [ ]:
toronto_venues = getNearbyVenues(names = toronto_neighborhoods['Neighborhood'],
                                   latitudes = toronto_neighborhoods['Latitude'],
                                   longitudes = toronto_neighborhoods['Longitude']
                                  )

In [ ]:
toronto_venues.head()

Preprocess data to show for each neighborhood the average quantity of each type of venue

In [ ]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood Name'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby(['Neighborhood Name']).mean().reset_index()
toronto_grouped.head()

Show for each neighborhood the most common types of venues

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood Name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Perform the KMeans clustering to segment the Toronto neighborhoods

In [ ]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

toronto_merged = toronto_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood').reset_index(drop = True)

toronto_merged # check the last columns!

In [ ]:
toronto_merged.shape

In [ ]:
toronto_neighborhoods.shape

Mapping with different colors for each cluster the n-hoods

In [ ]:
import folium

import matplotlib.cm as cm
import matplotlib.colors as colors

latitude = 43.6529
longitude = -79.3849

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters